In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
class Equipamento:
    def __init__(self, 
                 tipo,
                 marca,
                 modelo,
                 local, #Local de instalação
                 tensao =None,
                 dc = True,#Tipo de alimentação Corrente Continua ou Alternada, por padrão Continua DC 
                 instalador=None,
                 data_compra=None,
                 temp_garantia=3, #Tempo de garantia em meses
                 data_instalacao=None,):
        
        self._tipo = tipo
        self._marca = marca
        self._modelo = modelo
        self._local = local
        self._tensao = tensao
        self._dc = dc
        self._temp_garantia = temp_garantia

        if instalador:
            self._instalador = instalador
        else:
            self._instalador = None

        if data_instalacao:
            self._data_instalacao = datetime.strptime(data_instalacao, "%d-%m-%Y")
        else:
            self._data_instalacao = None

        if data_compra:
            self._data_compra = datetime.strptime(data_compra, "%d-%m-%Y")
        else:
            self._data_compra = None
        
        self._data_ultimo_defeito = None
        self._ultimo_defeito = None
        self._historico_manutencao = pd.DataFrame(columns=["Data", "Tec","Defeito"])

    #Getters e Setters
    def getTipoEquipamento(self):
        return self._tipo
    
    def getMarca(self):
        return self._marca
    
    def getModelo(self):
        return self._modelo
    
    def getLocal(self):
        return self._local
    def _setLocal(self,local):
        self._local = local

    def getTensao(self):
        return self._tensao
    def _setTensao(self,tensao):
        self._tensao = tensao

    def isDC(self):
        return self._dc
    
    def getTempGarantia(self):
        return self._temp_garantia
    
    def getInstalador(self):
        return self._instalador
    
    def _setInstalador(self,instalador):
        self._instalador = instalador
    
    def getCompra(self):
        if self._data_compra: 
            return self._data_compra
        else:
            return f"Não foi definida a data de compra do(a) {self.getTipoEquipamento()}"
    
    def getInstalacao(self):
        if self._data_instalacao:
            return self._data_instalacao
        else:
            return f"Não foi definida a data de instalação do(a) {self.getTipoEquipamento()}"
        
    def getDataUltimaManutencao(self):
        if self._data_ultima_manutencao:
            return self._data_ultima_manutencao
        else:
            return "não houve manutenções ainda"
        
    def getTecUltimaManutencao(self):
        if self._tec_ultima_manutencao:
            return self._tec_ultima_manutencao
        else:
            return "não houve manutenções ainda"
    
    def _setUltimoDefeito(self,data,desc_defeito):
        self._ultimo_defeito = desc_defeito
        self._data_ultima_medicao = data
    
    def IsForaDaGarantia(self):
        if self._data_compra:
            if self._data_compra + relativedelta(months=self.getTempGarantia()) < datetime.now():
                return True
            else:
                return False
        else:
            if self.getTempGarantia():
                return f"Não foi definida a data de compra do(a) {self.getTipoEquipamento()}"
            else:
                return "Não foi informado o tempo de garantia"
    
    def reinstalacao(self,instalador,local,data,tensao=None):
        self._data_instalacao = datetime.strptime(data, "%d-%m-%Y")
        self._setInstalador(instalador)
        self._setLocal(local)
        if tensao:
            self._setTensao(tensao)

    def manutencao(self,tec,data,defeito):
        self._data_ultima_manutencao = datetime.strptime(data, "%d-%m-%Y")
        self._tec_ultima_manutencao = tec
        if defeito:
            self._setUltimoDefeito(self.getDataUltimaManutencao(),defeito)
        nova_medicao = pd.DataFrame({'Data':[self.getDataUltimaManutencao()], 'Tec':[self.getTecUltimaManutencao()], "Defeito":[defeito]})
        self._historico_manutencao = pd.concat([self._historico_manutencao, nova_medicao], ignore_index=True)

    

In [3]:
class Bateria(Equipamento):
    def __init__(self, 
                 marca,
                 modelo,
                 local, #Local de instalação
                 medicao_inicial=12.8, 
                 tensao_ciclo=(13.5,13.8), 
                 tensao_carregamento=(14.3,14.8),
                 tensao =None,
                 dc = True,#Tipo de alimentação Corrente Continua ou Alternada, por padrão Continua DC 
                 instalador=None,
                 data_compra=None,
                 temp_garantia=3, #Tempo de garantia em meses
                 data_instalacao=None,
                 ):
        super().__init__(tipo="Bateria",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=None,
                         dc=True,
                         instalador=instalador,
                         data_compra=data_compra,
                         temp_garantia=temp_garantia,
                         data_instalacao=data_instalacao,
                         )
        
        self._tensao_ciclo = tensao_ciclo
        self._tensao_carregamento = tensao_carregamento
        self._tensao_ultima_manutencao = medicao_inicial
        self._data_ultima_manutencao = self._data_instalacao
        if self._data_ultima_manutencao:
            self._prox_medicao = self._data_ultima_manutencao + relativedelta(months=3)
        else:
            self._prox_medicao = None
        self._tensao_baixa = False
        self._historico_manutencao = pd.DataFrame(columns=['Data', 'Tensao'])

    def __str__(self):
        output=f"""Marca: {self.getMarca()}
Data de Compra: {self.getCompra()}
Data Instalação: {self.getInstalacao()}
Última Medição: {self.getDataUltimaManutencao()}
Tensão de Ciclo: {self.getTensaoCiclo()}
Tensão de Carregamento: {self.getTensaoCarregamento()}"""
        return output

    # Getters and Setters
    def getTensaoCiclo(self):
        return self._tensao_ciclo

    def getTensaoCarregamento(self):
        return self._tensao_carregamento

    def getTensao(self):
        return self._tensao_ultima_manutencao

    def IsLowVoltage(self):
        return self._tensao_baixa

    def getProxManutencao(self):
        if self._prox_manutencao:
            return self._prox_manutencao
        else:
            return "Não há data da última medição"

    def _setTensao(self, tensao):
        if tensao < self.getTensaoCiclo()[0]:
            self._tensao_baixa = True
        self._tensao_ultima_medicao = tensao

    def _proximaManutencao(self): # Adiciona 3 meses à data da última medicão
        self._prox_manutencao = self._data_ultima_manutencao + relativedelta(months=3)
        return self._prox_manutencao

    def manutencao(self, tec, data, tensao, defeito):
        super().manutencao(tec,data,defeito)
        self._setTensao(tensao)
        nova_medicao = pd.DataFrame({"Data":[self.getDataUltimaManutencao()], 
                                     "Tec":[self.getTecUltimaManutencao()], 
                                     "Defeito":[defeito], 
                                     "Tensao":[self.getTensao()]})
        self._historico_manutencao = pd.concat([self._historico_manutencao, nova_medicao], ignore_index=True)
        self._prox_manutencao = self._proximaManutencao()
        

In [129]:
class FonteTimer(Equipamento):
    def __init__(self,tec,local:str,marca:str,modelo:str,bateria:Bateria,tensao_saida=12,tensao_entrada=127,data_instalacao=None,temp_garantia=None,data_compra=None):
        super().__init__(tipo="FonteTimer",
                         marca=marca,
                         modelo=modelo,
                         local=local,
                         tensao=tensao_entrada,
                         dc=False,
                         instalador=tec,
                         data_compra=data_compra,
                         data_instalacao=data_instalacao,
                         temp_garantia=temp_garantia)
        
        self._bateria = bateria
        self._tensao_saida = tensao_saida
        self._ultima_tensao_saida = self._tensao_saida
        self._ultima_tensao_entrada = self._tensao

    def setBateria(self,bateria:Bateria):
        self._bateria = bateria
    def getBateria(self):
        return self._bateria
    
    def getTensaoSaida(self):
        return self._tensao_saida
    
    def getUltimaTensaoEntrada(self):
        return self._ultima_tensao_entrada
    
    def manutencao(self,tec,data,tensao_entrada=None, tensao_saida=None,defeito=None):
        if tensao_entrada: #Se a tensão de entrada foi medida na manutenção
            self._ultima_tensao_entrada = tensao_entrada
        if tensao_saida: #Se a tensão de saída foi medida na manutenção
            self._tensao_saida = tensao_saida
        tensao_ref = self.getTensao(self)
        tensao_medida = self.getUltimaTensaoEntrada(self)
        if tensao_ref - tensao_medida > (tensao_ref*0.07) : #Queda de tensão maxima permitida pela NBR5410
            defeito=f"Tensão de entrada ({tensao_medida} V) abaixo de {self.getTensao(self)*0.93}V (-7% do previsto)"
        super().manutencao(tec=tec,data=data,defeito=defeito)
            

In [130]:
import re
def valida_ipv4(ip): 
            regex = re.compile(r"^(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9]{2}|[1-9]?[0-9])$") 
            return regex.match(ip) is not None

class EquipamentoDeRede:
    def __init__(self,ip,mascara_sub_rede,gateway):
        if valida_ipv4(ip):
            self._ip = ip
        if valida_ipv4(mascara_sub_rede):
            self._mascara_sub_rede = mascara_sub_rede
        if valida_ipv4(gateway):
            self._gateway = gateway

    def getIp(self):
        return self._ip
    def setIp(self,ip):
        self._ip = valida_ipv4(ip)
    
    def getMascaraSubRede(self):
        return self._mascara_sub_rede
    def setMascaraSubRede(self,mascara_sub_rede):
        self._mascara_sub_rede = valida_ipv4(mascara_sub_rede)

    def getGateway(self):
        return self._gateway
    def setGateway(self,gateway):
        self._gateway = valida_ipv4(gateway) 
       
        

In [126]:
class Leitor(EquipamentoDeRede):
    def __init__(self,nome,marca,modelo,ip,mascara_sub_rede,gateway,RS485,):
        super().__init__(self,ip,mascara_sub_rede,gateway)
        self._nome = nome
        self._marca = marca
        self._modelo = modelo
        self._RS485 = RS485
     
    def getNome(self):
        return self._nome
    def setNome(self):
        self._nome 
    
    def getRS485(self):
        return self._RS485
    def setRS485(self,RS485):
        self._RS485 = RS485


In [ ]:
class BotoeiraEmergencia(Equipamento):
    def __init__(self,marca,modelo,data_compra, data_instalacao,temp_garantia):
        super().__init__()
        

In [127]:
class Porta:
    def __init__(self,nome,botoeira_emerg,ima,leitor_entrada,leitor_saida):
        self._nome = nome
        self._botoeira_emerg = botoeira_emerg
        self._ima = ima
        self._leitor_entrada = leitor_entrada
        self._leitor_saida = leitor_saida
    def getNome(self):
        return self._nome
    def setNome(self,nome):
        self._nome = nome
    def getBotoeira_emerg(self):
        return self._botoeira_emerg
    def setBoteira_emerg(self,botoeira):
        self._botoeira_emerg = botoeira
    def getIma(self):
        return self._ima
    def setIma(self,ima):
        self._ima = ima
    def getLeitorEntrada(self):
        return self._leitor_entrada
    def setLeitorEntrada(self,leitor):
        self._leitor_entrada = leitor
    def getLeitorSaida(self):
        if self._leitor_saida:
            return self._leitor_entrada
        else:
            return "Não existe leitor de saída"
    def setLeitorSaida(self,leitor):
        self._leitor_entrada = leitor


In [128]:
class Controladora(FonteTimer):
    def __init__(self,local:str,
                 marca:str,
                 modelo:str,
                 bateria:Bateria,
                 portas:list,
                 ip:str,
                 mascara_sub_rede:str,
                 gateway:str,
                 tensao_saida=12,
                 tensao_entrada=127,
                 data_instalacao=None,
                 temp_garantia=None,
                 data_compra=None,):
        super().__init__(local,marca,modelo,bateria,tensao_saida,tensao_entrada,data_instalacao,data_instalacao,temp_garantia,data_compra)
        self._portas = portas
        self._ip = ip
        self._mascara_sub_rede = mascara_sub_rede
        self._gateway = gateway
        


In [117]:

# Exemplo de uso
bateria = Bateria(marca="Intelbras")
print(bateria.getTensaoCiclo())  # (13.5, 13.8)

(13.5, 13.8)


In [119]:
fonteTimer_1 = FonteTimer("4GAB","Intelbras",bateria)

In [123]:
fonteTimer_1.getBateria().getUltimaMedicao()

datetime.datetime(2025, 1, 10, 0, 0)

In [116]:
bateria.getUltimaMedicao()

In [122]:
bateria.medicao(tensao=12.1,data="10-01-2025")

In [112]:
bateria.IsForaDaGarantia()

True

In [111]:
bateria._data_compra = datetime.strptime("09-09-2023","%d-%m-%Y")